In [4]:
!python -m pip install ase cloudpickle dscribe nglview

  Using cached dscribe-1.2.2.tar.gz (1.2 MB)


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\ge47yej\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ge47yej\\AppData\\Local\\Temp\\pip-install-yy5hlc0n\\dscribe_fa64d2d21f4245a0b9a79425e2a31414\\setup.py'"'"'; __file__='"'"'C:\\Users\\ge47yej\\AppData\\Local\\Temp\\pip-install-yy5hlc0n\\dscribe_fa64d2d21f4245a0b9a79425e2a31414\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ge47yej\AppData\Local\Temp\pip-wheel-44ikjxdj'
       cwd: C:\Users\ge47yej\AppData\Local\Temp\pip-install-yy5hlc0n\dscribe_fa64d2d21f4245a0b9a79425e2a31414\
  Complete output (69 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\dscribe
  copying dscribe\__init__.py -> build\lib.win-amd64-3.8\d

  Running setup.py clean for dscribe
Failed to build dscribe
    Running setup.py install for dscribe: started
    Running setup.py install for dscribe: finished with status 'error'


  copying dscribe\ext\soapGTO.cpp -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\soapGeneral.cpp -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\weighting.cpp -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\acsf.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\celllist.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\cm.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\descriptor.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\descriptorglobal.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\geometry.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\mbtr.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\soap.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\soapGTO.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\soapGeneral.h -> build\lib.win-amd64-3.8\dscribe\ext
  copying dscribe\ext\weighting.h -> build\lib.win-amd64-3.8\

In [3]:
from functools import partial

import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import cloudpickle
import json

# plotting
import matplotlib.pyplot as plt

import importlib as imp
import sklearn_utils
imp.reload(sklearn_utils)
from sklearn_utils import run_regressor_nested_cv

import ase
import ase.io

import dscribe

ModuleNotFoundError: No module named 'dscribe'

In [ ]:
data = pd.read_excel("data/dataset_pp.2022-12-09.xlsx", index_col=0)
for folder in [p for p in data["path"] if not os.path.exists(os.path.join(p, "POSCAR"))]:
    os.rename(os.path.join(folder, "POSCAR.vasp"), os.path.join(folder, "POSCAR"))
[p for p in data["path"] if not os.path.exists(os.path.join(p, "POSCAR"))]

In [ ]:
data = pd.read_excel("data/dataset_pp.2022-12-09.xlsx", index_col=0)
data["structure"] = [ase.io.read(os.path.join(p, "POSCAR")) for p in data["path"]]

extra_features = data["System"].str.split("-")
extra_df = pd.DataFrame.from_dict(dict(zip(extra_features.index, extra_features.values))).T
extra_df.columns = ["...", "sys", "pos", "ads"]
extra_df = extra_df.iloc[:, 1:]
ads_names = extra_df["ads"]
ads_cats = ads_names.astype("category").cat.codes


from dscribe.descriptors import SOAP
soap_sub = SOAP(species=["Mg"], periodic=True, r_cut=2, n_max=2, l_max=2,
                average="outer")

soap_ads = SOAP(species=["C", "H", "O", "Mg"], periodic=True, r_cut=4.0, n_max=4, l_max=3,
                average="outer")


In [ ]:
allfp_sub = []
allfp_ads = []

for s in data["structure"].to_list():
    substrate_index = np.array(s.get_chemical_symbols()) == "Mg"
    substrate = s[substrate_index]
    adsorbant = s[np.logical_not(substrate_index)]
    sub_z = substrate.positions[:,2].min()
    ads_C = adsorbant.positions[adsorbant.numbers == 6][0]
    ads_C_z = ads_C[2]
    ads_ref = ads_C.copy()
    ads_ref[2] -= (ads_C_z - sub_z)
    ads_ref_atom = ase.Atoms("Mg", [ads_ref,])
    ads_fp_sys = adsorbant + ads_ref_atom
    
    # substrate fp
    fp_sub = soap_sub.create([substrate])
    # full structure fp around the adsorbant
    #fp_sub = soap_ads.create([s], [[ads_ref,]])
    # reference system only
    fp_ads = soap_ads.create([ads_fp_sys], [[ads_ref],])
    
    allfp_sub.append(fp_sub)
    allfp_ads.append(fp_ads)
    
allfp_sub = np.array(allfp_sub)
allfp_ads = np.array(allfp_ads)

total_fp = np.hstack([allfp_sub, allfp_ads])
target = data["E_ads"]
print()
from ase.visualize import view
print(ads_C, ads_ref, allfp_sub.shape, allfp_ads.shape, total_fp.shape)

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR

from sklearn.decomposition import PCA

total_fp_pca = PCA(n_components=20).fit_transform(total_fp)

res = run_regressor_nested_cv(total_fp, target, 
                        partial(KernelRidge), dict(
                         alpha = np.logspace(-3, 4, num=20),
                         kernel= ["rbf"],
                         gamma = np.logspace(-3, 4, num=20),
                        ),
                        sample_class=ads_cats,
                        view_class=ads_names,
                        test_split=0.2,
                        scaler=None, name="SOAP_KRR",
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

In [ ]:

from sklearn.ensemble import RandomForestRegressor

total_fp_pca = PCA(n_components=20).fit_transform(total_fp)

res = run_regressor_nested_cv(total_fp, target, 
                        partial(RandomForestRegressor, random_state=0), dict(
                            n_estimators = np.logspace(1, 3.5, num=5).astype(int),
                            max_depth = [None, 5, 10, 20],
                            min_samples_split = [2, 5, 10]
                        ),
                        #sample_class=ads_cats,
                        view_class=ads_names,
                        test_split=0.2,
                        scaler=None, name="SOAP_RF",
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

In [ ]:
import sklearn_utils
imp.reload(sklearn_utils)
from sklearn_utils import run_regressor_nested_cv, run_regressor_manual
## FULL DATA GBR
from sklearn.ensemble import GradientBoostingRegressor


res = run_regressor_nested_cv(total_fp, target, 
                        partial(GradientBoostingRegressor, random_state=0), dict(
                            n_estimators = np.logspace(1, 2.5, num=5).astype(int),
                            max_depth = [1, 5, 6],
                            min_samples_split = [2, 3, 4]),
                        sample_class=ads_cats,
                        view_class=ads_names,
                        scaler=StandardScaler, name="SOAP_GB_base",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)